# Pytorch CNN Playground

In [68]:
import torch.nn as nn
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from common.base_model import BaseModel
import gym
import matplotlib
import matplotlib.pyplot as plt

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [69]:
class Model(BaseModel): 
    def __init__(self):
        print ("CNN Model Init")
    
    def add(self, state, action, reward, next_state, done):
        pass

    def sample(self, *args, **kwargs):
        pass

    def saveModel(self, *args, **kwargs):
        pass

    def loadModel(self, *args, **kwargs):
        pass
    
def get_model():
    return Model()
    

## Create Environment

In [70]:
env = gym.make('CartPole-v0').unwrapped
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()

## Setup Replay Mem

In [74]:
Transition = namedtuple("Transition", ('state', 'action', 'next_state', 'reward'))
class ReplayMem:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = [None for _ in range(self.capacity)]
        self.position = 0
    
    def push(self, trans):
        idx = self.position+1 % self.capacity
        self.memory[idx] = trans
    
    def sample(self, num_samples):
        return random.sample(self.memory, num_samples)

    def __len__(self):
        return len(self.memory)
        

## Create DQN Network

In [100]:
def size_finder(w, kernal_size,stride,padding=0):
    return ((w-kernal_size)/stride)+1

class NN(nn.Module):
    '''
    Image comes in:
        - 84x84x4 (stack 4 pics)
        - c1 => 20x20x32
        - c2 => 9x9x64
        - c3 => 7 x 7 x 64
        
        - fc input is flattened => 7*7*64 => 3136
        - output maps 256 inputs to n_actions as outputs
    '''
    def __init__(self,n_actions):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            
            
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
        
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        
        self.fc = nn.Sequential(nn.Linear(7*7*64, 512),
                                nn.ReLU(),
                                nn.Linear(512, n_actions)
                               )
        
    def forward(self, x):
        conv_output = self.conv(x)
        flatten = conv_output.view(x.size()[0], -1)
        return self.fc(flatten)
        

16